In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import pprint
import itertools
import pickle
import sklearn
import dask
import dask.dataframe as dd
import dask.array as da
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random

### Analysing similarity to own MSA-PC patent field vs other similar patent fields
- For new technology fields, analyse similarity to own MSA-PC patent field, vs other MSA-PCs
    - Can be: (i) in same MSA, different PC; (ii) in different MSA, same PC; (iii) in different MSA, different PC
    - There's a lot of traction for (ii) having pairs more similar
    
1. Identify primary classes that only reached over 20 patents per year after 1990, but with total patents greater than 1000
2. Identify MSA-PC subfields with those primary classes and the first year they received more than 5 patents
3. 

In [58]:
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas()
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]
# Add MSA subfield
pdf["msa_pc"] = list(zip(pdf["inv_msa"], pdf["primclass"]))

In [83]:
# New technology fields
pc_size = pdf["primclass"].value_counts()
pc_size = pc_size[pc_size > 1000]

# Identify first year in which primary class exceeded 20 patents
N = 20
p2 = pdf.loc[pdf["primclass"].isin(pc_size.index), ["primclass", "gyear", "patent"]]\
.groupby(["primclass", "gyear"]).size()
p2 = p2.loc[(p2 >= N)].reset_index().rename(columns={0:"size"})

# First year the field reaches more than N total patents
p2 = p2[["primclass", "gyear"]].drop_duplicates(subset=["primclass"], keep="first")
pc_first = p2.loc[p2["gyear"] >= 1985]

In [91]:
# # Get size of msa_pc by year
p2 = pdf[["inv_msa", "primclass", "gyear", "patent"]].groupby(["inv_msa", "primclass", "gyear"]).size()
p2 = p2.loc[(p2 >= 5)].reset_index().rename(columns={0:"size"})

# Check number of years available
p3 = p2.groupby(["inv_msa", "primclass"]).size()
# Places with more than 10 years of 5 patents available
p3 = p3.loc[(p3 >= 10)].reset_index().rename(columns={0:"size"}).drop("size",1)

In [92]:
# First gyear
p3 = p3.merge(p2[["inv_msa", "primclass", "gyear"]].drop_duplicates(subset=["inv_msa", "primclass"], keep="first"),
              how="left", on=["inv_msa","primclass"]).rename(columns={"gyear": "first_gyear"})
print(len(p3))

# Only use new primary classes
p3 = p3.loc[p3["primclass"].isin(pc_first["primclass"])]
print(len(msa_pc_first))

p3["msa_pc"] = list(zip(p3["inv_msa"], p3["primclass"]))

2662
168


In [94]:
# tp: patent from new subfield
# Primary class random sample
ns = fastparquet.ParquetFile("DataStore/2018-07-P2/Reg0726/reg_naics_name_sim_tr_0726.parq").to_pandas()
ns["op_msa_pc"] = list(zip(ns["op_inv_msa"], ns["op_primclass"]))
ns["tp_msa_pc"] = list(zip(ns["tp_inv_msa"], ns["tp_primclass"]))
ns = ns.loc[ns["tp_msa_pc"].isin(p3["msa_pc"])]

# Find highly similar PC subfields not from the same MSA
ns2 = ns.loc[(ns["norm_sim_mean_docvecs_pc_msa"] >= 1.5) &
             (ns["inv_msa_match"] == False), ["tp_msa_pc", "op_msa_pc",
"tp_primclass", "op_primclass", "tp_inv_msa", "op_inv_msa", "inv_msa_match", "year_group", 
"sim_mean_docvecs_pc_msa", "norm_sim_mean_docvecs_pc_msa"]].drop_duplicates()

# Comparison groups
ns3 = ns2[["tp_msa_pc", "op_msa_pc"]]
ns3["tp_first_gyear"] = ns3["tp_msa_pc"].map(dict(zip(p3["msa_pc"], p3["first_gyear"])))
print(len(ns3))

2155


/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [97]:
# Use only patents from relevant MSA-PC
p2 = pdf.loc[pdf["msa_pc"].isin(ns3["tp_msa_pc"]) | pdf["msa_pc"].isin(ns3["op_msa_pc"])]

# Patents by year
%time pats_by_yr = {n:g["patent"].tolist() for n,g in p2[["msa_pc", "gyear", "patent"]].groupby(["msa_pc", "gyear"])}

CPU times: user 1.58 s, sys: 570 µs, total: 1.58 s
Wall time: 1.59 s


In [98]:
# Add a grant year for each of the 10 years following the first grant year of new subfield
ns4 = [list(zip([r1]*10, [r2]*10, [r3]*10, list(range(r3, r3+10)))) for r1,r2,r3\
       in zip(ns3["tp_msa_pc"], ns3["op_msa_pc"], ns3["tp_first_gyear"])]
ns4 = [item for sublist in ns4 for item in sublist]
print(len(ns4))
# Same for self
ns5 = [list(zip([r1]*10, [r2]*10, [r3]*10, list(range(r3, r3+10)))) for r1,r2,r3\
       in zip(ns3["tp_msa_pc"], ns3["tp_msa_pc"], ns3["tp_first_gyear"])]
ns5 = [item for sublist in ns5 for item in sublist]

ns4.extend(ns5)
print(len(ns4))

ns4 = pd.DataFrame({"tp_msa_pc": [i[0] for i in ns4],
                   "op_msa_pc": [i[1] for i in ns4],
                   "tp_first_gyear": [i[2] for i in ns4],
                   "gyear": [i[3] for i in ns4],})

21550
43100


In [99]:
# Patents for each MSA-PC
ns4["tp_msa_pc_pats"] = [pats_by_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["tp_msa_pc"], ns4["gyear"])]
ns4["op_msa_pc_pats"] = [pats_by_yr.get((mp, yr), np.nan) for mp, yr in zip(ns4["op_msa_pc"], ns4["gyear"])]
print(len(ns4))
# Drop whereever any of the patent lists are missing
ns4 = ns4.dropna(subset=['tp_msa_pc_pats', 'op_msa_pc_pats',],how="any")
print(len(ns4))

43100
41297


In [100]:
ns4.to_pickle("DataStore/2018-08/new_related_field_pats_0918.pkl")

In [104]:
# Get sample pairs for each
%time msa_pc_pats = [list(itertools.product(i,j)) for i,j in zip(ns4["tp_msa_pc_pats"], ns4["op_msa_pc_pats"])]
# Add identifiers
%time msa_pc_pats = [list(zip(i, [(j,k,l,m)]*len(i))) for i,j,k,l,m in \
                    zip(msa_pc_pats, ns4["tp_msa_pc"], ns4["op_msa_pc"], ns4["gyear"], ns4["tp_first_gyear"])]
# Sample half
%time msa_pc_pats = (random.sample(l, int(np.round(len(l)/5))) for l in msa_pc_pats)
%time msa_pc_pats = [item[0]+item[1] for sublist in msa_pc_pats for item in sublist]


d = pd.DataFrame({"tp": [i[0] for i in msa_pc_pats],
                           "op": [i[1] for i in msa_pc_pats],
                            "tp_msa_pc": [i[2] for i in msa_pc_pats],
                            "op_msa_pc": [i[3] for i in msa_pc_pats],
                            "gyear": [i[4] for i in msa_pc_pats],
                            "tp_first_gyear": [i[5] for i in msa_pc_pats],})
d = d.drop_duplicates()
d = d.loc[~(d["tp"] == d["op"])]

# Drop same patent
rs = d
print(len(rs))
del(d)

CPU times: user 5.65 s, sys: 1.16 s, total: 6.81 s
Wall time: 6.85 s
CPU times: user 20 s, sys: 1.25 s, total: 21.3 s
Wall time: 21.4 s
CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 10.7 µs
CPU times: user 21.2 s, sys: 2.09 s, total: 23.3 s
Wall time: 22.8 s
2921115


In [105]:
# Get similarity
def grouper(n, iterable):
    """
    >>> list(grouper(3, 'ABCDEFG'))
    [['A', 'B', 'C'], ['D', 'E', 'F'], ['G']]
    """
    iterable = iter(iterable)
    return iter(lambda: list(itertools.islice(iterable, n)), [])


import scipy.spatial.distance as distance
dms = ["ldavecs", "docvecs"]

print("Getting row values")
print(datetime.datetime.now())
pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
pat_dict = dict(zip(pat_dict, range(len(pat_dict))))

l2 = rs.drop_duplicates()

for dm in dms:
    print((dm,"started"))
    print("Loading matrix and dict")
    print(datetime.datetime.now())
    
    print(len(l2))
    # Store copy as array
    l3 = l2.loc[l2["tp"].isin(pat_dict.keys()) & l2["op"].isin(pat_dict.keys()), ["tp", "op"]].copy().drop_duplicates()

    if dm == "ldavecs":
        ncols = 60
    else:
        ncols = 100

    pm = fastparquet.ParquetFile("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm))\
.to_pandas().values

    # Convert to chunks
    print("Getting chunks")
    print(datetime.datetime.now())
    # Split into chunks
    n_rows = 3000
    n_chunks = int(np.round(len(l3)/n_rows))
    tp_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["tp"].iteritems()]])
    op_chunks = grouper(n_rows, pm[[pat_dict[p[1]] for p in l3["op"].iteritems()]])
    del(pm)
    chunks = itertools.zip_longest(tp_chunks, op_chunks)

    print("Getting patent pair cosine similarity")
    print(datetime.datetime.now())
    # Cosine

    cos_dis = np.empty(len(l3))

    for r, c in enumerate(chunks):
        cos_dis[r*n_rows:r*n_rows+n_rows] = np.diag(distance.cdist(c[0],c[1], metric = "cosine"))

    l3["sim_{0}".format(dm)] = 1-cos_dis
    del(cos_dis)

    # Rename columns
    l2 = l2.merge(l3.drop_duplicates(), how = "left", on = ["tp", "op"])
    del(l3)

    print("finished")
    print(datetime.datetime.now())
    

Getting row values
2018-09-18 17:47:07.818841
('ldavecs', 'started')
Loading matrix and dict
2018-09-18 17:47:10.791939
2921115
Getting chunks
2018-09-18 17:47:22.129614
Getting patent pair cosine similarity
2018-09-18 17:47:32.864066
finished
2018-09-18 17:52:41.302214
('docvecs', 'started')
Loading matrix and dict
2018-09-18 17:52:41.302348
2921115
Getting chunks
2018-09-18 17:53:00.849520
Getting patent pair cosine similarity
2018-09-18 17:53:20.013967
finished
2018-09-18 18:01:11.839995


In [49]:
l2.head()

,gyear,op,op_msa_pc,tp,tp_first_gyear,tp_msa_pc,sim_docvecs,sim_ldavecs
0,2001,6295526,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6317751,1996,"(New York-Northern New Jersey-Long Island, NY-...",0.132798,0.394436
1,2002,6415281,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6408295,1996,"(New York-Northern New Jersey-Long Island, NY-...",0.293622,0.233587
2,2002,6370523,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6341277,1996,"(New York-Northern New Jersey-Long Island, NY-...",0.245861,0.058523
3,2003,6671703,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6662181,1996,"(New York-Northern New Jersey-Long Island, NY-...",0.179977,0.263423
4,2004,6772151,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6785672,1996,"(New York-Northern New Jersey-Long Island, NY-...",0.291113,0.160807


In [47]:
rs.head()

,gyear,op,op_msa_pc,tp,tp_first_gyear,tp_msa_pc
0,2001,6295526,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6317751,1996,"(New York-Northern New Jersey-Long Island, NY-..."
1,2002,6415281,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6408295,1996,"(New York-Northern New Jersey-Long Island, NY-..."
2,2002,6370523,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6341277,1996,"(New York-Northern New Jersey-Long Island, NY-..."
3,2003,6671703,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6662181,1996,"(New York-Northern New Jersey-Long Island, NY-..."
4,2004,6772151,"(Atlanta-Sandy Springs-Marietta, GA, 707.0)",6785672,1996,"(New York-Northern New Jersey-Long Island, NY-..."


In [106]:
rs = l2
rs.to_pickle("DataStore/2018-08/new_field_pats_pairsim_0918.pkl")
del(l2)

In [107]:
r1 = rs[["tp_msa_pc", "op_msa_pc", "gyear", "sim_docvecs"]]\
.groupby(["tp_msa_pc", "op_msa_pc", "gyear"]).mean().reset_index().rename(columns={"sim_docvecs": "msa_pc_docvecs"})
ns4 = ns4.merge(r1, how="left", on = ["tp_msa_pc", "op_msa_pc", "gyear"])

In [108]:
ns4.to_pickle("DataStore/2018-08/new_related_field_pats_0918.pkl")